## Importing packages needed and read files from data-ID and emotion.csv
## Notice: 
1.Using pd.read_json to read 'tweets_DM.json', remember using the right orientation

2.json_normalize for extracting the nest structures(_source) to 2 levels

3.finally we sort using the 'tweet.tweet_id' and save them to 'df1', and pass it to the data's new column 'text'
## Using pd.query to distinguish Test and Train dataset and sort them with tweet_id

In [2]:

import pandas as pd
import numpy as np
from collections import Counter
import nltk
import pandas as pd
#from emoticons import EmoticonDetector
import re as regex
import numpy as np
#import plotly
#from plotly import graph_objs
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, RandomizedSearchCV
from time import time
import matplotlib.pyplot as plt
%matplotlib inline
#plotly.offline.init_notebook_mode()

import seaborn as sns
import plotly
import cufflinks as cf
import re
nltk.download('punkt')

data=pd.read_csv('data_identification.csv')
emotion1=pd.read_csv('emotion.csv')

emotion1=emotion1.sort_values(by=['tweet_id'])
emotion1.head()



data=data.sort_values(by=['tweet_id'])


emotion1=emotion1.sort_values(by=['tweet_id'])
emotion1


from pandas.io.json import json_normalize
df=pd.read_json("tweets_DM.json",lines='true',orient='columns')
df1=pd.json_normalize(df['_source'],max_level=2)



df1=df1.sort_values(by=['tweet.tweet_id'])
df1

df1
data['text']=df1['tweet.text']
data['text']
train=data.query('identification=="train"')
train=train.sort_values(by='tweet_id')
test=data.query('identification=="test"')
train
test=test.sort_values(by='tweet_id')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ken\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Using sort at first according to the twitter_id, then add a new column index for train dataset

In [3]:
train['index']=emotion1['emotion'].values


## Make a function for tweets text cleaning
re.sub can replace the redundant symbols of tweets to null:

In [4]:

def clean_tweets(tweet):
    
    # remove URL
    tweet = re.sub(r"http\S+", "", tweet)
    
    # Remove usernames
    tweet = re.sub(r"@[^\s]+[\s]?",'',tweet)
    
    # remove special characters 
    tweet = re.sub('[^ a-zA-Z0-9]', '', tweet)
    
    # remove Numbers
    tweet = re.sub('[0-9]', '', tweet)
    
    return tweet

## Using apply to clean train['text'] and test['text']

In [5]:
train['text'] = train['text'].apply(clean_tweets)
test['text'] = test['text'].apply(clean_tweets)

In [6]:

import nltk
from sklearn.feature_extraction.text import CountVectorizer




## I personally found some LH symbols that seem to be redudant and clean them

In [7]:
train['text']=train['text'].apply(lambda x: x.replace("LH",""))
test['text']=test['text'].apply(lambda x: x.replace("LH",""))


## importing training requirements

In [11]:
import numpy as np
from  tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.sequence import pad_sequences
import os
import pandas as pd
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report



## combi(Train+Test) used for w2v training

In [12]:
combi = train.append(test, ignore_index=True, sort=True)
combi.shape

(1867535, 4)

In [37]:
train['index'].value_counts()
#using these frequencies afterwards

joy             516017
anticipation    248935
trust           205478
sadness         193437
disgust         139101
fear             63999
surprise         48729
anger            39867
Name: index, dtype: int64

In [13]:
def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)
    return input_txt

## combi cleaning

In [14]:
combi['tidy_tweet'] = np.vectorize(remove_pattern)(combi['text'], "@[\w]*") 
combi.head(10)

,identification,index,text,tweet_id,tidy_tweet
0,train,joy,thanks for waking me up God,0x1c7f10,thanks for waking me up God
1,train,anticipation,With expectation there you will find anticipat...,0x1c7f11,With expectation there you will find anticipat...
2,train,joy,glad has discovered Rochdale Borough last week...,0x1c7f14,glad has discovered Rochdale Borough last week...
3,train,joy,Only down We need The D to step up and just h...,0x1c7f15,Only down We need The D to step up and just h...
4,train,disgust,I want to an Insecure recap video with a guy o...,0x1c7f16,I want to an Insecure recap video with a guy o...
5,train,anticipation,OMGNo no no What What No you cant leave it lik...,0x1c7f19,OMGNo no no What What No you cant leave it lik...
6,train,joy,look at this guy youre not a cowboy Boomer,0x1c7f1a,look at this guy youre not a cowboy Boomer
7,train,anticipation,Airlines in America SAY one thing in disingenu...,0x1c7f1b,Airlines in America SAY one thing in disingenu...
8,train,trust,roosterpisces janetshoemake MargaretSteib Mar...,0x1c7f1c,roosterpisces janetshoemake MargaretSteib Mar...
9,train,joy,We hope your Halloween was a happy one and wit...,0x1c7f1d,We hope your Halloween was a happy one and wit...


In [15]:
combi.tidy_tweet = combi.tidy_tweet.str.replace("[^a-zA-Z#]", " ")
combi.head(10)

,identification,index,text,tweet_id,tidy_tweet
0,train,joy,thanks for waking me up God,0x1c7f10,thanks for waking me up God
1,train,anticipation,With expectation there you will find anticipat...,0x1c7f11,With expectation there you will find anticipat...
2,train,joy,glad has discovered Rochdale Borough last week...,0x1c7f14,glad has discovered Rochdale Borough last week...
3,train,joy,Only down We need The D to step up and just h...,0x1c7f15,Only down We need The D to step up and just h...
4,train,disgust,I want to an Insecure recap video with a guy o...,0x1c7f16,I want to an Insecure recap video with a guy o...
5,train,anticipation,OMGNo no no What What No you cant leave it lik...,0x1c7f19,OMGNo no no What What No you cant leave it lik...
6,train,joy,look at this guy youre not a cowboy Boomer,0x1c7f1a,look at this guy youre not a cowboy Boomer
7,train,anticipation,Airlines in America SAY one thing in disingenu...,0x1c7f1b,Airlines in America SAY one thing in disingenu...
8,train,trust,roosterpisces janetshoemake MargaretSteib Mar...,0x1c7f1c,roosterpisces janetshoemake MargaretSteib Mar...
9,train,joy,We hope your Halloween was a happy one and wit...,0x1c7f1d,We hope your Halloween was a happy one and wit...


## if length of the words>3, x.join

In [16]:
combi.tidy_tweet = combi.tidy_tweet.apply(lambda x: ' '.join([w for w in x.split() if len(w) > 3]))
combi.head(10)

,identification,index,text,tweet_id,tidy_tweet
0,train,joy,thanks for waking me up God,0x1c7f10,thanks waking
1,train,anticipation,With expectation there you will find anticipat...,0x1c7f11,With expectation there will find anticipation ...
2,train,joy,glad has discovered Rochdale Borough last week...,0x1c7f14,glad discovered Rochdale Borough last week tod...
3,train,joy,Only down We need The D to step up and just h...,0x1c7f15,Only down need step just hang there keep Faith...
4,train,disgust,I want to an Insecure recap video with a guy o...,0x1c7f16,want Insecure recap video with YouTube channel...
5,train,anticipation,OMGNo no no What What No you cant leave it lik...,0x1c7f19,OMGNo What What cant leave like this LethalWeapon
6,train,joy,look at this guy youre not a cowboy Boomer,0x1c7f1a,look this youre cowboy Boomer
7,train,anticipation,Airlines in America SAY one thing in disingenu...,0x1c7f1b,Airlines America thing disingenuous campaigns ...
8,train,trust,roosterpisces janetshoemake MargaretSteib Mar...,0x1c7f1c,roosterpisces janetshoemake MargaretSteib Mari...
9,train,joy,We hope your Halloween was a happy one and wit...,0x1c7f1d,hope your Halloween happy with just right amount


In [17]:
tokenized_tweet = combi.tidy_tweet.apply(lambda x: x.split())
tokenized_tweet.head()

0                                     [thanks, waking]
1    [With, expectation, there, will, find, anticip...
2    [glad, discovered, Rochdale, Borough, last, we...
3    [Only, down, need, step, just, hang, there, ke...
4    [want, Insecure, recap, video, with, YouTube, ...
Name: tidy_tweet, dtype: object

## Using stemming technique referred in class

In [18]:
from nltk.stem.porter import * 
stemmer = PorterStemmer() 
tokenized_tweet = tokenized_tweet.apply(lambda x: [stemmer.stem(i) for i in x]) # stemming
tokenized_tweet.head()

0                                        [thank, wake]
1    [with, expect, there, will, find, anticip, rev...
2    [glad, discov, rochdal, borough, last, week, t...
3    [onli, down, need, step, just, hang, there, ke...
4    [want, insecur, recap, video, with, youtub, ch...
Name: tidy_tweet, dtype: object

In [19]:
for i in range(len(tokenized_tweet)):
    tokenized_tweet[i] = ' '.join(tokenized_tweet[i])    
combi['tidy_tweet'] = tokenized_tweet
combi.head(10)

,identification,index,text,tweet_id,tidy_tweet
0,train,joy,thanks for waking me up God,0x1c7f10,thank wake
1,train,anticipation,With expectation there you will find anticipat...,0x1c7f11,with expect there will find anticip revel thur...
2,train,joy,glad has discovered Rochdale Borough last week...,0x1c7f14,glad discov rochdal borough last week today cu...
3,train,joy,Only down We need The D to step up and just h...,0x1c7f15,onli down need step just hang there keep faith...
4,train,disgust,I want to an Insecure recap video with a guy o...,0x1c7f16,want insecur recap video with youtub channel b...
5,train,anticipation,OMGNo no no What What No you cant leave it lik...,0x1c7f19,omgno what what cant leav like thi lethalweapon
6,train,joy,look at this guy youre not a cowboy Boomer,0x1c7f1a,look thi your cowboy boomer
7,train,anticipation,Airlines in America SAY one thing in disingenu...,0x1c7f1b,airlin america thing disingenu campaign piss s...
8,train,trust,roosterpisces janetshoemake MargaretSteib Mar...,0x1c7f1c,roosterpisc janetshoemak margaretsteib marizel...
9,train,joy,We hope your Halloween was a happy one and wit...,0x1c7f1d,hope your halloween happi with just right amount


## Use bow_vectorizer to fit_transform combi['tidy_tweet']

In [20]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer 
import gensim

In [21]:
combi.reset_index(drop=True)
bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')
bow = bow_vectorizer.fit_transform(combi['tidy_tweet'])
bow.shape

(1867535, 1000)

## Model_w2v training process:
    
    1.prepare tokenized text
    2.settings fits(vector_size,window_size,ignore_count,skip-gram,hs,negative sampling,no.cores,seed)
    3.train the model with tokenized text, using epochs=20

In [23]:
%%time

tokenized_tweet = combi['tidy_tweet'].apply(lambda x: x.split()) # tokenizing 

model_w2v = gensim.models.Word2Vec(
            tokenized_tweet,
            vector_size=200, # desired no. of features/independent variables
            window=5, # context window size
            min_count=2, # Ignores all words with total frequency lower than 2.                                  
            sg = 1, # 1 for skip-gram model
            hs = 0,
            negative = 10, # for negative sampling
            workers= 32, # no.of cores
            seed = 34
) 

model_w2v.train(tokenized_tweet, total_examples= len(combi['tidy_tweet']), epochs=20)

Wall time: 10min 25s


(253704890, 287562000)

## function of tokens to word_vector process

In [24]:
def word_vector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0
    for word in tokens:
        try:
            vec += model_w2v.wv[word].reshape((1, size))
            count += 1.
        except KeyError:  # handling the case where the token is not in vocabulary
            continue
    if count != 0:
        vec /= count
    return vec

## After putting values using loop, we construct a dataframe to save it in and check its shape

In [25]:
wordvec_arrays = np.zeros((len(tokenized_tweet), 200)) 
for i in range(len(tokenized_tweet)):
    wordvec_arrays[i,:] = word_vector(tokenized_tweet[i], 200)
wordvec_df = pd.DataFrame(wordvec_arrays)
wordvec_df.shape

(1867535, 200)

In [26]:
from tqdm import tqdm 
tqdm.pandas(desc="progress-bar") 
from gensim.models.doc2vec import TaggedDocument

## Import taggeddocoument to add_label function

In [27]:
def add_label(twt):
    output = []
    for i, s in zip(twt.index, twt):
        output.append(TaggedDocument(s, ["tweet_" + str(i)]))
    return output

labeled_tweets = add_label(tokenized_tweet) # label all the tweets

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

## Transform emotion string to number to train in xgb-classfier

In [38]:
def  emotion_num(emotion):
        if  emotion=='joy':
            return 0
        
        if  emotion=='anticipation':  
            return 1
        if   emotion=='trust':     
            return 2
        if emotion=='sadness':
            return 3
        if emotion=='disgust':
            return 4
        if emotion=='fear':
            return 5
        if emotion=='surprise':
            return 6
        if emotion=='anger':
            return 7

In [39]:
train['index']=train['index'].apply(emotion_num)

## Reset_index due to the sort of tweet_id and divide bow into train_bow and test_bow


In [40]:
train=train.reset_index(drop=True)
from xgboost import XGBClassifier
train_bow = bow[:1455563,:] 
test_bow = bow[1455563:,:] 

# splitting data into training and validation set 
xtrain_bow, xvalid_bow, ytrain, yvalid = train_test_split(bow[:100000], train['index'][:100000], random_state=42, test_size=0.3)

## if we don't reset train_index above, we'll get errors when we put ytrain.index and yvalid.index in, due to the no. would possibly exceed the limit of the train_w2v matrix

In [44]:
train_w2v = wordvec_df.iloc[:1455563,:]
test_w2v = wordvec_df.iloc[1455563:,:]

xtrain_w2v = train_w2v.iloc[ytrain.index,:]
xvalid_w2v = train_w2v.iloc[yvalid.index,:]

In [46]:
xgb_model = XGBClassifier(max_depth=6, n_estimators=1000).fit(xtrain_bow, ytrain)
prediction = xgb_model.predict(xvalid_bow)
f1_score(yvalid, prediction ,average='micro')

[09:09:14] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


0.3323333333333333

## Trying Trying and Falling back


I transfer the number back into emotions and upload it to the competition. And i also try roberta training, but the grade 
was lower than this version.(Roberta cost me nearly about 2 weeks, so it's really disappointing that it's not working better) 

i work it using emotion predictor(unsupervised pretrained model learning)
and i use colab to run, debugging cost me about 90 percents of time. At last i finally run it but the test data is too large,
and i split about 410000 texts run inpendently with 42 folds each with 10000 texts.
Combining and Waiting, writing it to submission.csv but not good work...but it's a good experience
https://colab.research.google.com/drive/1QDtn3t2ZLAV5Gz4yfHYAsRkJpkORdivw?usp=sharing

Spent too long time running following codes(the trained wv model above), a little bit upset when waiting all day long without any improvements, hoping to give more
advices if you want.

In [ ]:
xgb = XGBClassifier(max_depth=6, n_estimators=1000, nthread= 3,use_label_encoder=False).fit(xtrain_w2v, ytrain) 
prediction = xgb.predict(xvalid_w2v)
f1_score(yvalid, prediction,average='micro')

[09:13:46] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


## Future Expectation and My experience in this competition
Cause the imcompletion of the execution, i have no time to do hyper-tuning and model-esembling.
Really happy to do such codeworks in this course. Though i always fall behind the class progress, and fail the orientation test.
I believe i've done my best in nfl analysis and the final exam.  Data-analysist is my dream job and this is the first time.
Thanks teachers and tas for solving problems with students actively and without leaving any rest encouraging us to have new 
thoughts about data-mining. If there is any chance for me to attend the alike works again, i'll have some basic data-mining
coding ability. Thanks a lot again, and hoping that i can still ask questions about this region. Really nice course and i will
recommend to my classmates to join as well.

In [ ]:
import xgboost as xgb

In [ ]:
dtrain = xgb.DMatrix(xtrain_w2v, label=ytrain) 
dvalid = xgb.DMatrix(xvalid_w2v, label=yvalid) 
dtest = xgb.DMatrix(test_w2v)
# Parameters that we are going to tune 
params = {
    'objective':'binary:logistic',
    'max_depth':6,
    'min_child_weight': 1,
    'eta':.3,
    'subsample': 1,
    'colsample_bytree': 1
 }